In [30]:
import random
from draftsman.blueprintable import Blueprint
from draftsman.entity import AssemblingMachine, Inserter, Entity
from draftsman.constants import Direction



In [72]:
def generate_factory_grid(n_assemblers: int, grid_size=20):

    recipes_pool = ["iron-gear-wheel", "automation-science-pack", "electronic-circuit"]

    bp = Blueprint()
    placed_positions = set()
    occupied = set()

    # Add ee-infinity-loader source and sink
    try:
        source_loader = Entity("ee-infinity-loader", similar_entities=[], tile_position=(0, 0), direction=Direction.EAST)
        sink_loader = Entity("ee-infinity-loader", similar_entities=[], tile_position=(grid_size - 2, grid_size - 2), direction=Direction.WEST)

        # Set a filter on the sink to distinguish it
        # sink_loader.filters = ["iron-plate"]  # example filter

        bp.entities.extend([source_loader, sink_loader])
    except Exception as e:
        print(f"Could not add ee-infinity-loaders: {e}")

    # Place assemblers with inserters
    while len([e for e in bp.entities if isinstance(e, AssemblingMachine)]) < n_assemblers:
        x = random.randint(1, grid_size - 4)
        y = random.randint(1, grid_size - 4)

        # Decide vertical or horizontal using boolean
        is_vertical = random.choice([True, False])
        assembler_dir = Direction.SOUTH if is_vertical else Direction.EAST

        # 3x3 assembler occupies (x,y) to (x+2,y+2)
        body_tiles = {(x + dx, y + dy) for dx in range(3) for dy in range(3)}

        # Inserter tiles (input and output) depend on orientation
        if is_vertical:
            in_pos = (x + 1, y + 3)   # Bottom center
            out_pos = (x + 1, y - 1)  # Top center
        else:
            in_pos = (x - 1, y + 1)   # Left center
            out_pos = (x + 3, y + 1)  # Right center

        all_needed_tiles = body_tiles.union({in_pos, out_pos})
        if any(tile in occupied or not (0 <= tile[0] < grid_size and 0 <= tile[1] < grid_size) for tile in all_needed_tiles):
            continue  # Retry if overlapping or out of bounds

        assembler = AssemblingMachine("assembling-machine-1", position=(x + 1, y + 1), direction=assembler_dir)
        assembler.recipe = random.choice(recipes_pool)

        if is_vertical:
            inserter_in = Inserter(position=in_pos, direction=Direction.SOUTH)
            inserter_out = Inserter(position=out_pos, direction=Direction.SOUTH)
        else:
            inserter_in = Inserter(position=in_pos, direction=Direction.SOUTHEAST)
            inserter_out = Inserter(position=out_pos, direction=Direction.SOUTHEAST)

        try:
            bp.entities.extend([assembler, inserter_in, inserter_out])
            occupied.update(all_needed_tiles)
        except ValueError:
            continue


        try:
            bp.entities.extend([assembler, inserter_in, inserter_out])
            placed_positions.update([(x, y), (x, y + 1), (x, y - 1)])
        except ValueError:
            continue

    

    return bp


def generate_blueprint_string(n_assemblers: int, grid_size=20):
    bp = generate_factory_grid(n_assemblers, grid_size)
    return bp.to_string()  # Convert blueprint to a base64 string

In [74]:
# Example usage:
for i in range(0,3):
    blueprint_string = generate_blueprint_string(4, 20)
    print("Blueprint String:", blueprint_string)

Blueprint String: 0eNrF09tqhDAQBuB3mWtTjMaoeZWlFHWn7VATxcRSEd+9cdsFWyvFBdmrDDl8yfyQEcq6x7Yj40CNQA41qMVcAO/YWWoMqERGucjzRPI0ziIRABpHjtCCOo1gCo3+JCIj80yG3MDqpjhj54m2sX7jbIzwASp8CAMYLuMUwJk6rL5Wr+bwZHpd+qOKT8Eummff9lz8xHm00qOFXliLuqzJvDBdVK9kkPG1L6+8/M1nAfia2ovVu0YX8zyzFaGpkLVF9QarB8SLB1jnt/gOLXbur9bkZmfZyhW3uOJ/N7l3YPKgwNKDAst2B8bTTV7ckli+ozOebN4t118z3CPne2R+99R4dFhs8WGxielxmj4BNtDo2Q==
Blueprint String: 0eNrFkN1ugzAMRt/F12Qi/JNXqaYpUK+1RgJKwjaEePeFjkmdaKdxgXoVK7HPF58RqqbHzpB2IEYghwrE1V0A72gstRpEmkVlUpZpxvO4iJIAUDtyhBbEYQQtFfpJREb6lTS5gTWtPKLxiK61vnFmjPAJInwKAxgu5xTAkQzW368/zOFF96ryo4JPwSY0Lxb2XPyG82hFj67o0lpUVUP6xJSsz6SR8TU/XvD5ja/7mroLqnetkvM9szWhrpF1sn6DVX58lW+db/ELWjTu5mb3krMVNtmATf+PTR9sK9vHVr6PrWKzrWyBx3/ZIuM9nVAa9nFGbNaWyg3rJPcS1+vwcAO32MDlD/LEo51ExTuJSqbnafoCYabeow==
Blueprint String: 0eNrFk9tugzAMQP/Fz2Qi4VLgV6ZpgtTbrJGAkjANIf59oWsrVpi2VkJ9iuU4x/aRMkBVd9ga0g6KAcihgmKWC+ADjaVGQ5GkIo/zPEn5LspEHABqR47QQvE4gC4V+peIjPQLaXI9q5tyj8Yj2sb6wokxwCcU4UMYQH84xwD2ZFB+356Y/bPuVOWfFnwMrkLz7Mieg

C:\Users\selin\AppData\Local\Temp\ipykernel_6620\3609738444.py:63: OverlappingObjectsWarning: Added object 'assembling-machine-1' (AssemblingMachine) at (6.0, 16.0) intersects 'assembling-machine-1' (AssemblingMachine) at (6.0, 16.0)
  bp.entities.extend([assembler, inserter_in, inserter_out])
C:\Users\selin\AppData\Local\Temp\ipykernel_6620\3609738444.py:63: OverlappingObjectsWarning: Added object 'stack-inserter' (Inserter) at (6.0, 18.0) intersects 'stack-inserter' (Inserter) at (6.0, 18.0)
  bp.entities.extend([assembler, inserter_in, inserter_out])
C:\Users\selin\AppData\Local\Temp\ipykernel_6620\3609738444.py:63: OverlappingObjectsWarning: Added object 'stack-inserter' (Inserter) at (6.0, 14.0) intersects 'stack-inserter' (Inserter) at (6.0, 14.0)
  bp.entities.extend([assembler, inserter_in, inserter_out])
C:\Users\selin\AppData\Local\Temp\ipykernel_6620\3609738444.py:63: OverlappingObjectsWarning: Added object 'assembling-machine-1' (AssemblingMachine) at (17.0, 14.0) intersect